# MacroServer and its elements
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents

* MacroServer overview
* Macro features
* Generic Scan Framework
* Recorders

## [MacroServer overview](http://www.sardana-controls.org/en/latest/devel/overview/overview_macroserver.html)

* Is a controlled environment to run procedures, called macros, sequentially or simultaneously
* Provides a standard catalogue of procedures both user and expert
* Allows to plug in new/custom procedures written as Python functions or classes (advanced)
* Connects to device Pool(s) and allows to act on their elements from within macros

## [Macro features](http://www.sardana-controls.org/en/latest/devel/howto_macros/macros_general.html)

* Add example macros, in spock: `_MACRO_SERVER.put_property({"MacroPath":["/sardana/src/sardana/macroserver/macros/examples"]})`
* Restart Sardana server

### Macro parameters

* Benefits
 * The parameters validation prevents runtime errors
 * Spock offers autocompletion facilities
 * Macro execution widgets offers list of allowed parameter values in combo boxes
 * Autogenerated documentation

### Macro parameters

* Parameters are defined either as arguments of the `macro` decorator for the macro function or as `param_def` class member for the macro class
* A parameter is characterized by: name, type, default value and description
* Parameter values arrives as positional arguments to the macro function or to the run method of the macro class

### Macro parameters - demo

* `prdef pt1`
* `pt1 1.0`
* `prdef pt2`
* `pt2 mot01`

### Macro parameters - repeat parameters

* Allows to pass as parameter value a list of repetitions of the repeat parameter member(s) - an empty list is also allowed, meaning no values
* Repeat parameters allow to:
 * restrict the minimum and/or maximum number of repetitions
 * nest repeat parameters inside of another repeat parameters
 * define multiple repeat parameters in the same macro


### Macro parameters - repeat parameters - demo

* `prdef pt4`
* `pt4 mot01 mot02`
* `prdef pt5`
* `pt5 mot01 1 2.5`


### Macro results

* Allows to pass the macro result to the clients (string representation) and to the wrapper macro
* Results are defined as the `result_def` class member of the macro class
* A result is characterized by: name, type, default value and description - exactly the same as macro parameter
* Multiple results are possible

### Macro results - demo

* `prdef twice`
* `twice 2`
* result: 4.0

### Logging in macros

* Macros may log messages to various destinations
 * Spock or other clients
 * MacroServer log files
* Different log levels are possible e.g. output, error, debug

### Macro data

* Allows to pass the macro data to the clients (serialized with `pickle`) and to the wrapper macro
* Data format is totally opened

### Macro data - demo

* Execute macro twice: `twice 3`
* Access macro data with `macrodata`
* Execute scan: `ascan mot01 0 10 2 0.1`
* Access macro data with `macrodata`

### Hooks

* Python code that will be executed at a given point of the macro
* Can be a Python callable or a macro
* Hooks can be attached to the hook places that defines macros
* Macro must inherit from `Hookable` class to allow hook places
* Hook places are defined using the `hints` class member (dictionary) and the `allowsHooks` key
* Exists two possibilities to attach hooks to the macros:
 * programatically - using the `hooks` macro attribute (it requires defining a wrapper macro)
 * graphically - using the `sequencer` widget


### Hooks - demo

* `prdef loop`
* `prdef captain_hook`
* `captain_hook 1 10 1`

### Nesting macros (a.k.a. wrapper macros)

* Macros can call other macros, and these other macros can call another macros, and so on...
* Many different ways exists to execute other macros from within a macro:
 * each macro is available as a macro class method e.g. `ascan`, `lsm`
 * `createMacro` and `runMacro` (create calls the macro's `prepare` method and run calls the macro's `run` method) - may be useful if we want to manipulate the macro object e.g. attach hooks
 * `execMacro` - `prepare` and `run` called at once

### Nesting macros (a.k.a. wrapper macros) - demo

* See previous example: `prdef captain_hook`
* `prdef mfunc1`
* `mfunc1`

### Interactive macros

* Macros can ask for user input
* Macro must be declared as interactive e.g. `imacro` decorator or `interactive` class member set to `True`
* The user input request may be characterized with: type, default value, title, unit, label, range, ...
* Spock may handle user input request either in the CLI or GUI mode - see `SPOCK_INPUT_HANDLER` in the `sardana.sardanacustomsettings`

### Interactive macros - demo

* `prdef ask_number_of_points`
* `ask_number_of_points`
* `prdef ask_for_moveable`
* `ask_for_moveable`

### Ploting in macros

* Macro send request to plot to the client e.g. spock
* Uses `matplotlib.pyplot` framework API e.g. `pyplot.plot`

### Ploting in macros - demo

* `prdef random_image`
* `random_image`

### Interrupting macro execution

* Macros  can be stopped, aborted or paused
* As soon as the user requested the macro interruption the macro will get interrupted on the next call of `checkPoint` (or `pausePoint`) or any of the macro API method (Macro class methods decorated with `mAPI`)
* Long sleeps should be interleaved with check points
* Macro may define `on_stop`, `on_abort` and `on_pause` method that will be called on the corresponding interruption
* Macro reserved object, for example requested with the `getMotion` method (`getMotor` will not reserve the motor object!), are stopped/aborted on the corresponding interruption (before calling the `on_stop`/`on_abort` method)

### Reporting macro progress

* Macros may report their progress while are executed
* The progress is usually expressed in percentage 0 - 100 %
* The progress update is done using the `yield` statement within the `run` method

### Reporting macro progress - demo

* Be patient, and wait for macroexecutor demo...

## Generic Scan Framework

* Scanning modes: step, continuous (pure software - c, hardware and software - ct), hybrid and time
* Only the ct-mode continuous scans use events to transfer data, the rest of them read the channel values
* Provides the catalogue of turn-key scan macros e.g. `ascan`, `a2scanc`, `ascanct`, `mesh`, `timescan`, etc.
* Provides the base classes for developing custom scans e.g. `GScan`, `SScan`, `CScan`, etc.
* Custom scans are developed by instantiating the base class and developing a custom generator
* Scans are highly configurable with the [environment variables](http://www.sardana-controls.org/en/latest/users/scan.html#configuration) e.g. `ActiveMntGrp`, `ScanFile`, `ApplyInterpolation`
* Scans macros [support hooks](http://www.sardana-controls.org/en/latest/devel/howto_macros/scan_framework.html#hooks-support-in-scans) e.g. `pre-scan`, `post-acq`

## Generic Scan Framework - demo

* Start taurustrend with high frequency of polling: `taurustrend mot01/position mntgrp13/state ct01/state -r 10`
* Change curve titles in taurus trend to show `<dev_alis>`
* Demonstrate timescan: `timescan 10 1 0.05`
* Demonstrate step scan: `ascan mot01 0 10 10 1`
* Demonstrate continuous scan: `ascan mot01 0 10 10 1 0.05`

## [Recorders](http://www.sardana-controls.org/en/latest/devel/howto_recorders.html#writing-recorders)

![Sardana data flow](res/recorders.png)